# 2.2 Programming an eigenvalue solver

We solve the generalized eigenvalue problem

$$
A u = \lambda M u,
$$

where $A$ comes from $\int \nabla u \nabla v$, and $M$ from $\int u v$, on the space $H_0^1$.

This tutorial shows how to implement linear algebra algorithms.

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
import math
import scipy.linalg
from scipy import random

mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))


We setup a stiffness matrix $A$ and a mass matrix $M$, and declare a preconditioner for $A$:

In [2]:
fes = H1(mesh, order=3, dirichlet=".*")
print ("ndof =", fes.ndof)
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
pre = Preconditioner(a, "multigrid")

m = BilinearForm(fes)
m += u*v*dx

a.Assemble()
m.Assemble()

u = GridFunction(fes)

ndof = 1096


The inverse iteration is

$$
u_{n+1} = A^{-1} M u_n,
$$

where the Rayleigh quotient
$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left 
\langle M u_n, u_n\right \rangle}
$$

converges to the smallest eigenvalue $\lambda_1$, with rate of convergence $\lambda_1 / \lambda_2,$ where $\lambda_2$ is the next smallest eigenvalue.

The preconditioned inverse iteration (PINVIT), see [Knyazef+Neymeyr], replaces $A^{-1}$ by an approximate inverse $C^{-1}$:

$$
\rho_n = \frac{\left \langle A u_n, u_n\right \rangle}{\left \langle M u_n, u_n\right \rangle} \\
w_n = C^{-1} (A u_n - \rho_n M u_n) \\
u_{n+1} = u_n + \alpha w_n
$$

The optimal step-size $\alpha$ is found by minimizing the Rayleigh-quotient on a two-dimensional space:

$$
u_{n+1} = \operatorname{arg} \min_{v \in \{ u_n, w_n\}} \frac{\left \langle A v, v\right \rangle}{\left \langle M v, v\right \rangle} 
$$

This minimization problem can be solved by a small eigenvalue problem

$$
a y = \lambda m y
$$

with matrices

$$
a = 
\left( \begin{array}{cc}
\left \langle  A u_n, u_n \right \rangle &
\left \langle  A u_n, w_n \right \rangle \\
\left \langle  A w_n, u_n \right \rangle &
\left \langle  A w_n, w_n \right \rangle 
\end{array} \right), \quad
m = 
\left( \begin{array}{cc}
\left \langle  M u_n, u_n \right \rangle &
\left \langle  M u_n, w_n \right \rangle \\
\left \langle  M w_n, u_n \right \rangle &
\left \langle  M w_n, w_n \right \rangle 
\end{array} \right).
$$

Then, the new iterate is

$$
u_{n+1} = y_1 u_n + y_2 w_n
$$

where $y$ is the eigenvector corresponding to the smaller eigenvalue.

### Implementation in NGSolve

First, we create some help vectors. `CreateVector` creates new vectors of the same format as the existing vector, i.e., same dimension, same real/ complex type, same entry-size, and same MPI-parallel distribution if any.

In [3]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

Next, we pick a random initial vector, which is  zeroed on the Dirichlet boundary.

Below, the `FV` method (short for `FlatVector`)  lets us access the abstract vector's  linear memory chunk, which in turn provides a "numpy" view of the memory. The projector clears the entries at the Dirichlet boundary:

In [4]:
import numpy as np
r.FV().NumPy()[:] = random.rand(fes.ndof)
r.FV().NumPy()[:] = np.ones(fes.ndof)
u.vec.data = Projector(fes.FreeDofs(), True) * r

Finally, we run the PINVIT algorithm. Note that the small matrices $a$ and $m$ defined above are called `asmall` and `msmall` below. They are of type `Matrix`, a class provided by  NGSolve for  dense matrices.

In [5]:
for i in range(1):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotient
    lam = auu/muu
#     print (lam / (math.pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)
    
    
    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    print (eval)
    print(eval[0])
    print(eval[1])
    print(evec)
    print(evec[0,:])
    print(evec[1,:])
#     u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
    
# Draw (u)
# print(u.vec.data)
# print(float(evec[0,0]))

[ 20.21769289 365.91291915]
20.217692891713213
365.91291915033173
[[-0.3027279   2.52309058]
 [12.98689001 30.04924763]]
[-0.3027279   2.52309058]
[12.98689001 30.04924763]


## Simultaneous iteration for several eigenvalues

Here are the steps for extending the above to `num` vectors.

Declare a `GridFunction` with multiple components to store several eigenvectors:

In [6]:
num = 2
u = GridFunction(fes, multidim=num)

Create list of help vectors, and a set of random initial vectors in `u`, with zero boundary conditions:

In [7]:
r = u.vec.CreateVector()
Av = u.vec.CreateVector()
Mv = u.vec.CreateVector()

vecs = []
for i in range(2*num):
    vecs.append (u.vec.CreateVector())

for v in u.vecs:
    r.FV().NumPy()[:] = random.rand(fes.ndof)
    v.data = Projector(fes.FreeDofs(), True) * r

Compute `num` residuals, and solve a small eigenvalue problem on a 2 $\times$ `num` dimensional space:

In [8]:
asmall = Matrix(2*num, 2*num)
msmall = Matrix(2*num, 2*num)
lams = num * [1]

for i in range(20):
    
    for j in range(num):
        vecs[j].data = u.vecs[j]
        r.data = a.mat * vecs[j] - lams[j] * m.mat * vecs[j]
        vecs[num+j].data = pre.mat * r

    for j in range(2*num):
        Av.data = a.mat * vecs[j]
        Mv.data = m.mat * vecs[j]
        for k in range(2*num):
            asmall[j,k] = InnerProduct(Av, vecs[k])
            msmall[j,k] = InnerProduct(Mv, vecs[k])

    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams[:] = ev[0:num]
#     vec = Matrix ( evec [: , 0: num ])
    
    print (i, ":", [lam/math.pi**2 for lam in lams])
#     print (i, ":", [lam/math.pi**2 for lam in lams])
    
    for j in range(num):
        u.vecs[j][:] = 0.0
        for k in range(2*num):
            u.vecs[j].data += float(evec[k,j]) * vecs[k]
            
Draw (u)

0 : [14.004501601219301, 85.13813074630346]
1 : [2.044852310012606, 11.54684002261811]
2 : [2.0011168316696923, 5.465668665604279]
3 : [2.000101216937072, 5.050970872955965]
4 : [2.0000116570553823, 5.009015318146094]
5 : [2.0000014735449527, 5.00187157639909]
6 : [2.0000002589769035, 5.000401330377869]
7 : [2.0000001116519592, 5.000088608860985]
8 : [2.0000000935881954, 5.000021427011097]
9 : [2.0000000913608527, 5.000006927135953]
10 : [2.000000091084994, 5.000003789390193]
11 : [2.000000091050724, 5.000003109064371]
12 : [2.0000000910464553, 5.0000029613577865]
13 : [2.0000000910459206, 5.0000029292508685]
14 : [2.000000091045856, 5.000002922265484]
15 : [2.000000091045848, 5.000002920744522]
16 : [2.000000091045847, 5.000002920413245]
17 : [2.000000091045847, 5.000002920340913]
18 : [2.0000000910458455, 5.000002920325065]
19 : [2.0000000910458473, 5.000002920321731]


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

The *multidim-component* select in the *Visualization* dialog box allows to switch between the components of the multidim-GridFunction.

## Implementation using MultiVector

The simultaneous iteration can be optimized by using `MultiVector`s introduced in NGSolve 6.2.2007. These are arrays of vectors of the same format. You can think of a MultiVector with m components of vector size n as an $n \times m$ matrix. 

  - a `MultiVector` consisting of num vectors of the same format as an existing vector vec is created via
MultiVector(vec, num). 
  - we can iterate over the components of a MultiVector, and the bracket operator allows to access a subset of vectors
  - linear operator application is optimized for MultiVector
  - vector operations are optimized and called as mv * densematrix: 
    $x = y * mat$ results in x[i] = sum_j y[j] * mat[j,i]  (where x and y are 'MultiVector's, and mat is a dense matrix) 
  - pair-wise inner products of two MultiVectors is available, the result is a dense matrix: InnerProduct(x,y)[i,j] =
  - mv.Orthogonalize() uses modified Gram-Schmidt to orthogonalize the vectors. Optionally, a matrix defining the inner product can be provided.
  - with mv.Append(vec) we can add another vector to the array of vectos. A new vector is created, and the values are copied
  - mv.AppendOrthogonalize(vec) appends a new vector, and orthogonalizes it against the existing vectors, which are assumed to be orthogonal. 


In [9]:
uvecs = MultiVector(u.vec, num)
vecs = MultiVector(u.vec, 2*num)

for v in vecs[0:num]:
    v.SetRandom()
uvecs[:] = pre * vecs[0:num]
lams = Vector(num * [1])

In [10]:
for i in range(20):
    vecs[0:num] = a.mat * uvecs - (m.mat * uvecs).Scale (lams)
    vecs[num:2*num] = pre * vecs[0:num]
    vecs[0:num] = uvecs
        
    vecs.Orthogonalize() # m.mat)

    asmall = InnerProduct (vecs, a.mat * vecs)
    msmall = InnerProduct (vecs, m.mat * vecs)
    
    ev,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    lams = Vector(ev[0:num])
    print (i, ":", [l/math.pi**2 for l in lams])

    uvecs[:] = vecs * Matrix(evec[:,0:num])

0 : [138.21976195167335, 503.60077035136624]
1 : [2.0097837881050533, 12.980358295150994]
2 : [2.0001778122781677, 5.476126863514855]
3 : [2.000010879218834, 5.082321410272602]
4 : [2.000001072359394, 5.015753381267591]
5 : [2.0000001922720765, 5.003192732403233]
6 : [2.0000001022637512, 5.00065596944959]
7 : [2.000000092346106, 5.000138008958483]
8 : [2.000000091200212, 5.00003124471081]
9 : [2.000000091064476, 5.00000917054459]
10 : [2.0000000910481193, 5.00000458578219]
11 : [2.0000000910461257, 5.000003632209192]
12 : [2.0000000910458824, 5.000003433308855]
13 : [2.000000091045849, 5.000003391784764]
14 : [2.0000000910458486, 5.000003383097999]
15 : [2.0000000910458446, 5.00000338127953]
16 : [2.0000000910458477, 5.0000033808982085]
17 : [2.0000000910458455, 5.00000338081832]
18 : [2.000000091045847, 5.000003380801494]
19 : [2.000000091045845, 5.000003380797813]


The operations are implemented using late evaluation. The operations return expressions, and computation is done within the assignment operator. The advantage is to avoid dynamic allocation. An exception is InnerProduct, which allows an expression in the second argument (and then needs vector allocation in every call).